# Split & Sort

This notebook takes 150x150px images of maps and allows a human to review them and tag where players for each team are present. The images pulled out by this notebook will be used to train the locator model, which identifies which grid on the map contains a hero.

In [1]:
import os
import PIL
PIL.PILLOW_VERSION = PIL.__version__
import fastai
from fastai.basic_train import load_learner
from fastai.vision import *
from tqdm import tqdm_notebook
from datetime import datetime

random.seed(datetime.now())

In [2]:
def empty_dir(folder):
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
        except Exception as e:
            print(e)

# Source Minimaps

Helpers that saturate the blue and red to make players more visible. They also make a white border around the image to help the model focus on players in the center of the image.

In [3]:
def colour_filter(img):
    converter = PIL.ImageEnhance.Color(img)
    img = converter.enhance(3)
    overlay = PIL.Image.new('RGBA', img.size, (0, 0, 0, 0))
    draw = PIL.ImageDraw.Draw(overlay)
    fill = (255,255,255,128)
    draw.rectangle((0,0,7,30), fill=fill)
    draw.rectangle((0,23,30,30), fill=fill)
    draw.rectangle((23,30,30,0), fill=fill)
    draw.rectangle((30,0,0,7), fill=fill)
    out = PIL.Image.alpha_composite(img, overlay)
    out = out.resize((90,90))
    return out

def colour_blast(src, dst):
    img = PIL.Image.open(src).convert("RGBA")
    img = colour_filter(img)
    
    img.save(dst)

## Stats

In [4]:
data_dir = "./locator/data/"

red_dir =  "./locator/data/red/"
blue_dir =  "./locator/data/blue/"
purple_dir = "./locator/data/purple/"
terrain_dir = "./locator/data/terrain/"

# player_src =  "./locator/data/raw-player/"
player_src = "./locator/data/cherry-pick/"
terrain_src =  "./locator/data/unsorted-terrain/"

l = len(os.listdir(red_dir))
print("Red: " + str(l))
l = len(os.listdir("./locator/data/unsorted-red/"))
print("Red Unsorted: " + str(l))
l = len(os.listdir(blue_dir))
print("Blue: " + str(l))
l = len(os.listdir("./locator/data/unsorted-blue/"))
print("Blue Unsorted: " + str(l))
l = len(os.listdir(purple_dir))
print("Terrain: " + str(l))

l = len(os.listdir(terrain_src))
print("Terrain Unsorted: " + str(l))
l = len(os.listdir(player_src))
print("Player Unsorted: " + str(l))

Red: 2616
Red Unsorted: 5187
Blue: 3071
Blue Unsorted: 4575
Terrain: 768
Terrain Unsorted: 0
Player Unsorted: 0


## Sort

Sort allows a human to go through a list of 30x30 images and label them as containing red, blue or no players.

In [5]:
import tkinter as tk
import csv
import random
from PIL import Image, ImageDraw, ImageTk
import shutil


source_dir = player_src
# source_dir = terrain_dir


def next_img(dest):
    root.moved_images.append(dest)
    root.img_index += 1
    draw_map()
    img = root.image_list[root.img_index]

def red():
    img = root.image_list[root.img_index]
    src = source_dir + img
    dest = red_dir + img
    os.rename(src, dest)
    next_img(dest)

def blue():
    img = root.image_list[root.img_index]
    src = source_dir + img
    dest = blue_dir + img
    os.rename(src, dest)
    next_img(dest)

def terrain():
    img = root.image_list[root.img_index]
    src = source_dir + img
    dest = terrain_dir + img
    os.rename(src, dest)
    next_img(dest)
    
def skip():
    next_img("")


def undo():
    img = root.moved_images.pop()
    name = img.split("/")[-1]
    root.img_index -= 1
    os.remove(img)
    draw_map()

def draw_map():
    img = Image.open(f"{source_dir}{root.image_list[root.img_index]}").convert('RGBA')
    converter = PIL.ImageEnhance.Color(img)   
    img2 = converter.enhance(2)

    overlay = Image.new('RGBA', img2.size, (255, 255, 255, 0))
    draw = ImageDraw.Draw(overlay)
    out = Image.alpha_composite(img2, overlay)
    root.photo = ImageTk.PhotoImage(out)
    map_label.configure(image=root.photo)
    root.photo2 = ImageTk.PhotoImage(img)
    map2_label.configure(image=root.photo2)


root = tk.Tk()
red_images = set(os.listdir(red_dir))
blue_images = set(os.listdir(blue_dir))
purple_images = set(os.listdir(purple_dir))
terrain_images = set(os.listdir(terrain_dir))
images = set(os.listdir(source_dir))
if len(images) != 0:
    images = list(images)
    images.sort()
    root.image_list = images
    root.img_index = 0
    root.moved_images = []

    map_label = tk.Label(root)
    map_label.pack()
    map2_label = tk.Label(root)
    map2_label.pack()
    draw_map()

    red_btn = tk.Button(root, text="Red", command=red, bg="red", height = 3, width = 30)
    red_btn.pack()
    blue_btn = tk.Button(root, text="Blue", command=blue, bg="blue", height = 3, width = 30)
    blue_btn.pack()
    terrain_btn = tk.Button(root, text="Terrain", command=terrain, bg="brown", height = 3, width = 30)
    terrain_btn.pack()

    skip_btn = tk.Button(root, text="Skip", command=skip, height = 3, width = 30)
    skip_btn.pack()
    undo_btn = tk.Button(root, text="Undo", command=undo, height = 3, width = 30)
    undo_btn.pack()

    root.mainloop()

# Fine Tune Review

This review pass runs the locator model, and allows a human to review the sorted images.

## Predict & Sort images

Run predictions on the images for review.

In [6]:
learn = load_learner("locator/data/final/all/models", "locator.pth")

In [159]:
def get_predictions(img_dir, learner, predictions):
    # Predict the grid image types
    test = ImageList.from_folder(img_dir)
    learner.data.add_test(test)
    preds = learner.get_preds(ds_type=DatasetType.Test)
    num_preds = len(preds)
    z = 0

    # Identify the grids which are player squares
    for i in range(len(preds[0])):
        p = preds[0][i].tolist()
        pred_map = {}
        for j in range(len(p)):
            pred_map[learn.data.classes[j]] = p[j]
        if pred_map["player"] < pred_map["terrain"] or pred_map["terrain"] > 0.3:
            continue
        if pred_map["red"] < pred_map["terrain"] and pred_map["blue"] < pred_map["terrain"]:
            continue
        if pred_map["red"] > pred_map["blue"]:
            category = "red"
        else:
            category = "blue"
        img = test.items[i]
        predictions.append((img.stem, category))

    return predictions

empty_dir("./locator/data/colour-blasted")
empty_dir("./locator/data/unsorted-blue")
empty_dir("./locator/data/unsorted-red")
empty_dir("./locator/data/unsorted-purple")
empty_dir("./locator/data/unsorted-terrain")
for f in os.listdir("./locator/data/raw-player"):
    if not ".png" in f:
        continue
    img = PIL.Image.open(f"./locator/data/raw-player/{f}").convert("RGBA")
    converter = PIL.ImageEnhance.Color(img)
    img = converter.enhance(3)
    img.save(f"./locator/data/colour-blasted/{f}")

predictions = get_predictions("./locator/data/colour-blasted/", learn, [])
predictions
for p in predictions:
    shutil.copy(f"./locator/data/raw-player/{p[0]}.png", f"./locator/data/unsorted-{p[1]}/{p[0]}.png")

## Cherry pick screenshots from maps

This helper allows humans to visualize predictions from the locator model and pick out images which should be labeled. This helps correct mistakes due to a hero not being present enough or misidentified because their hero color is bluish or reddish.

In [167]:
import tkinter as tk
import csv
import random
from PIL import Image, ImageDraw, ImageTk
import shutil

game = random.choice(os.listdir("./games"))
src_dir = f"./games/{game}/full/"

def draw_grid(draw, predictions):
    for p in predictions:
        if p[1] == "terrain":
            continue
        x = 15 * int(p[0])//1000
        y = 15 * (int(p[0])%1000)
        if p[1] == "purple":
            fill = (128,128,128,128)
        if p[1] == "blue":
            fill = (0,0,128,128)
        if p[1] == "red":
            fill = (128,0,0,128)
        draw.rectangle((x-7, y-7, x+7, y+7), fill=fill)

def next_img():
    draw_map()

def draw_map():
    game = random.choice(os.listdir("./games"))
    src_dir = f"./games/{game}/full/"
    images = os.listdir(src_dir)
    random.shuffle(images)
    root.image = f"{src_dir}/{random.choice(images)}"
    img = Image.open(root.image).convert('RGBA')
    empty_dir(data_dir+"/tmp")
    
    for i in range(0, img.height, 15):
        for j in range(0, img.width, 15):
            box = (j-15, i-15, j + 15, i + 15)
            a = img.crop(box)
            a = colour_filter(a)
            a.save(data_dir+f"/tmp/{j*1000//15+i//15}.png")
    
    predictions = get_predictions(data_dir+"/tmp", learn, [])

    overlay = Image.new('RGBA', img.size, (255, 255, 255, 0))
    draw = ImageDraw.Draw(overlay)
    draw_grid(draw, predictions)
    out = Image.alpha_composite(img, overlay)
    root.raw_img = img
    root.photo = ImageTk.PhotoImage(out)
    map_label.configure(image=root.photo)
    
def click(event):
    cropped = root.raw_img.crop((event.x-15, event.y-15, event.x+15, event.y+15))
    cropped.save(f"./locator/data/cherry-pick/{event.x}-{event.y}-{root.image.split('/')[-1]}")


root = tk.Tk()

map_label = tk.Label(root)
map_label.bind("<Button-1>", click)
map_label.pack()
draw_map()

next_btn = tk.Button(root, text="Next", command=next_img, height = 3, width = 30)
next_btn.pack()

root.mainloop()

# Sort Bottom/Top Team Maps

This helper allows a human to create a spreadshot of which games were recorded for a top team and which are for a bottom team. This will allow us to correct the fact that top is always red in the full map.

In [4]:
import os
import tkinter as tk
import csv
import random
from PIL import Image, ImageDraw, ImageTk
import shutil

games = os.listdir("./games")
games.sort()


def next_img():
    root.game_index += 1
    game = games[root.game_index]
    draw_map()
    
def top():
    root.writer.writerow([games[root.game_index], 'top'])
    root.game_index += 1
    game = games[root.game_index]
    draw_map()
    
def bottom():
    root.writer.writerow([games[root.game_index], 'bottom'])
    root.game_index += 1
    game = games[root.game_index]
    draw_map()

def draw_map():
    images = os.listdir(f"./games/{games[root.game_index]}/team")
    images.sort()
    img = Image.open(f"./games/{games[root.game_index]}/team/{images[150]}").convert('RGBA')
    img2 = Image.open(f"./games/{games[root.game_index]}/team/{images[300]}").convert('RGBA')
    root.photo = ImageTk.PhotoImage(img)
    root.photo2 = ImageTk.PhotoImage(img2)
    map_label.configure(image=root.photo)
    map2_label.configure(image=root.photo2)

root = tk.Tk()
root.game_index = 0

import csv
with open('./games/team_location.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    root.writer.writerow(["game", position])
    root.game_index = 0

    root.writer = writer 
    map_label = tk.Label(root)
    map2_label = tk.Label(root)
    map_label.pack()
    map2_label.pack()
    draw_map()

    top_btn = tk.Button(root, text="Top", command=top, height = 3, width = 30)
    top_btn.pack()
    bottom_btn = tk.Button(root, text="Bottom", command=bottom, height = 3, width = 30)
    bottom_btn.pack()

    next_btn = tk.Button(root, text="Next", command=next_img, height = 3, width = 30)
    next_btn.pack()

    root.mainloop()

ahq_dwg_1_worlds_oct_13_2019
ahq_dwg_1_worlds_oct_20_2019
ahq_ig_1_worlds_oct_12_2019
ahq_ig_1_worlds_oct_20_2019
ahq_tl_1_worlds_oct_20_2019
c9_g2_1_worlds_oct_18_2019
c9_grf_1_worlds_oct_15_2019
c9_grf_1_worlds_oct_18_2019
c9_hka_1_worlds_oct_13_2019
c9_hka_1_worlds_oct_18_2019
cg_fnc_1_worlds_oct_13_2019
cg_fnc_2_worlds_oct_19_2019
cg_mmm_worlds_2019
cg_rng_1_worlds_oct_12_2019
cg_rng_1_worlds_oct_19_2019
cg_ryl_1_worlds_2019
cg_ryl_2_worlds_2019
cg_ryl_3_worlds_2019
cg_skt_1_worlds_oct_15_2019
cg_skt_1_worlds_oct_19_2019
cg_uol_worlds_2019
dfm_spy_worlds_2019
dwg_fla_worlds_2019
dwg_ig_1_worlds_oct_20_2019
dwg_lk_2_worlds_2019
dwg_lk_3_worlds_2019
dwg_lk_4_worlds_2019
dwg_lk_worlds_2019
dwg_ryl_worlds_2019
dwg_tl_1_worlds_oct_12_2019
dwg_tl_1_worlds_oct_20_2019
fla_ryl_2_worlds_2019
fla_ryl_3_worlds_2019
fla_ryl_worlds_2019
fnc_rng_1_worlds_oct_13_2019
fnc_rng_1_worlds_oct_19_2019
fnc_skt_1_worlds_oct_12_2019
fnc_skt_1_worlds_oct_19_2019
fpx_gam_1_worlds_oct_15_2019
fpx_gam_1_world

Exception in Tkinter callback
Traceback (most recent call last):
  File "/home/isaac/anaconda3/envs/fastai/lib/python3.7/tkinter/__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-4-06099a2b15fe>", line 21, in top
    draw_map()
  File "<ipython-input-4-06099a2b15fe>", line 31, in draw_map
    images = os.listdir(f"./games/{games[root.game_index]}/team")
NotADirectoryError: [Errno 20] Not a directory: './games/team_location.csv/team'
Exception in Tkinter callback
Traceback (most recent call last):
  File "/home/isaac/anaconda3/envs/fastai/lib/python3.7/tkinter/__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-4-06099a2b15fe>", line 20, in top
    game = games[root.game_index]
IndexError: list index out of range
